In [9]:
import open3d as o3d
import numpy as np
import time

pcd_track_uuids = np.load("../results/pcd/000/000016/pcd.npy",  allow_pickle=True).item()
# print("pcd_track_uuids", type(pcd_track_uuids))
# print("pcd_track_uuids", pcd_track_uuids.keys())
# ['3098a954-d9d8-4b9f-85a2-5523dba98fad', '2f999c71-bfc0-405e-a620-574545ffb657']
# print("pcd_track_uuids", pcd_track_uuids['3098a954-d9d8-4b9f-85a2-5523dba98fad'].keys())
# [2]
points = pcd_track_uuids['2f999c71-bfc0-405e-a620-574545ffb657'][6]
# points = pcd_track_uuids['3098a954-d9d8-4b9f-85a2-5523dba98fad'][2]


# Visualize results
vis = o3d.visualization.Visualizer()
vis.create_window()
# Coordinate frame
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
vis.add_geometry(axis_pcd)

# Create points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_scan)

vis.add_geometry(pcd)
for frame_id, points_scan in points.items():
    try:
        pcd.points = o3d.utility.Vector3dVector(points_scan)
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.1)
        # vis.remove_geometry(pcd, reset_bounding_box=False)
    except:
        pass

vis.destroy_window()

In [ ]:

    # for i in range(len(obj_codes)):
    #     mesh = mesh_extractor.extract_mesh_from_code(obj_codes[i])
    #     mesh_o3d.vertices = o3d.utility.Vector3dVector(mesh.vertices)
    #     mesh_o3d.triangles = o3d.utility.Vector3iVector(mesh.faces)
    #     mesh_o3d.compute_vertex_normals()
        

    #     correction_transform = obj.pose_list[i]
    #     # mesh_optimized = copy.deepcopy(mesh_o3d)
    #     mesh_o3d.transform(np.linalg.inv(correction_transform))

    #     # mesh_in_object_frame = copy.deepcopy(mesh_optimized)
    #     # APPLY INVERSE TRANSFORMATION TO GET MESH IN OBJECT FRAME WITH CONSTANT PARITY
    #     mesh_o3d.transform(np.linalg.inv(T_constant_parity))

    #     # Write to mesh
    #     # o3d.io.write_triangle_mesh("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/from_scratch/optimized_meshes/car1/mesh_optimized" +"_" +str(i) + ".ply", mesh_optimized )


    #     # Visualize mesh 
    #     vis.update_geometry(mesh_o3d)
    #     # vis.add_geometry(car)
    #     vis.poll_events()
    #     vis.update_renderer()
    #     time.sleep(0.5)
    #     vis.run()
    # vis.destroy_window()